In [2]:
from ultralytics import YOLO
import cv2

In [7]:
def resize_image(image, max_width=640, max_height=640):
    h, w = image.shape[:2]
    if w > max_width or h > max_height:
        scale_w = max_width / w
        scale_h = max_height / h
        scale = min(scale_w, scale_h)
        new_w = int(w * scale)
        new_h = int(h * scale)
        image = cv2.resize(image, (new_w, new_h))
    return image


model = YOLO("my_model.pt")


image_path = "images/photo_2025-04-25_16-52-46.jpg"  
image = cv2.imread(image_path)


results = model.predict(source=image, conf=0.4, show=False)


class_colors = {
    'Good': (0, 255, 0),
    'Defective': (0, 0, 255)
}
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 1.5
thickness = 5


for r in results:
    for box in r.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        cls_id = int(box.cls[0])
        conf = float(box.conf[0])
        label_name = model.names[cls_id]
        label = f"{label_name} ({conf:.2f})"
        color = class_colors.get(label_name, (255, 255, 0))  

        
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)

        
        (text_w, text_h), _ = cv2.getTextSize(label, font, font_scale, thickness)
        cv2.rectangle(image, (x1, y1 - text_h - 10), (x1 + text_w, y1), color, -1)

        
        cv2.putText(image, label, (x1, y1 - 5), font, font_scale, (255, 255, 255), thickness)


image = resize_image(image, max_width=600, max_height=700)


cv2.imshow("Potato Detection", image)
cv2.waitKey(0)
cv2.destroyAllWindows()



0: 640x288 1 Sprouted Potato, 196.1ms
Speed: 3.3ms preprocess, 196.1ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 288)
